In [1]:
!pip3 install transformers>=4.32.0 optimum>=1.12.0
!pip3 install auto-gptq --extra-index-url https://huggingface.github.io/autogptq-index/whl/cu118/  # Use cu117 if on CUDA 11.7

Looking in indexes: https://pypi.org/simple, https://huggingface.github.io/autogptq-index/whl/cu118/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 22.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.3/168.3 kB 17.2 MB/s eta 0:00:00


In [2]:
from pathlib import Path
import pandas as pd
from tqdm import tqdm
import os
import torch

from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

## Read Data

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
BASE_DIR = Path("/content/drive/MyDrive/ITMO/RW'23")
DATA_DIR = BASE_DIR.joinpath("data")

FILE_NAME = "data_topic_name_distrib_llama_13B_4bit_type1.csv"

FILE = DATA_DIR.joinpath(FILE_NAME)

In [7]:
data = pd.read_csv(FILE, names=["text", "topic"], sep=";")
data.head(10)

,text,topic
0,population bullet arab israeli border village ...,"Sure, I'd be happy to help!The topic of the te..."
1,quote hell context death accept evil agree fee...,Sure! Here's the topic of the text you provide...
2,enjoy strong morning brew blend bitter smooth ...,Sure! Here's the topic of the text you provide...
3,worth packet waste splenda world horrible when...,Sure! Here's the topic of the text you provide...
4,pres administration official congress secretar...,Sure! Here is the topic of the text you provid...
5,bible religion turkish church belief truth fai...,Sure! Here's the topic of the text you provide...
6,quote context umich taking feel learn maybe ag...,"Sure, I'd be happy to help! Here's the topic o..."
7,israel israeli arab palestinian attack murder ...,Sure! Here is the topic of the text you provid...
8,butter peanut apple orange aware jelly poor gr...,Sure! Here's the topic of the text you provide...
9,work package receive surprised sample recipe f...,Sure! Here's the topic of the text you provide...


In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2381 entries, 0 to 2380
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    2381 non-null   object
 1   topic   2381 non-null   object
dtypes: object(2)
memory usage: 37.3+ KB


##Llama-13B

In [9]:
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


In [10]:
model_name_or_path = "TheBloke/Llama-2-13B-chat-GPTQ"
# To use a different branch, change revision
# For example: revision="main"
model = AutoModelForCausalLM.from_pretrained(model_name_or_path,
                                             device_map="auto",
                                             trust_remote_code=False,
                                             revision="main")

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/837 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/7.26G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

ValueError: Found modules on cpu/disk. Using Exllama or Exllamav2 backend requires all the modules to be on GPU.You can deactivate exllama backend by setting `disable_exllama=True` in the quantization config object

##Inference

In [ ]:
def generate_answer(prompt, model, device, temp, n_token):
    encoded = tokenizer(prompt, return_tensors="pt", add_special_tokens=False)
    model_input = encoded
    model_input = model_input.to(device)
    generated_ids = model.generate(
        **model_input, do_sample=True,
        max_new_tokens=n_token,
        temperature=temp,
        num_return_sequences=1,
        pad_token_id=tokenizer.eos_token_id,
        num_beams=4
        )
    decoded = tokenizer.batch_decode(generated_ids)

    return decoded

In [ ]:
def extract_substring(input_string):
    index = input_string.find("[/INST]")
    if index != -1:
        return input_string[index + len("[/INST]"):]
    else:
        return ""

In [ ]:
def few_shot_pipeline(model, system_prompt='', instruction='', sample='', device='cpu', temp=0.1, n_token=100):
    # Construct prompt
    prompt = f"""<s>[INST]<<SYS>> {system_prompt} <</SYS>> {instruction} TEXT: {sample} ANSWER: [/INST]"""
    # Generate answer
    answer = generate_answer(prompt, model, device, temp, n_token)
    # Extract the result
    # print(answer[0])
    result = extract_substring(answer[0]).strip()
    return result


[INST] << SYS >>{system_prompt} << /SYS >>
{prompt}[/INST]

####Type I

In [ ]:
prompt = """<s>[INST] <<SYS>> You are the assistant for text classification <<SYS>>
You will receive a TEXT, and you should answer "YES" or "NO" to the question: "Is it possible to determine a common topic for the TEXT or at least for the most part of the TEXT?". Please, make sure you to only return YES or NO and nothing more.
TEXT: april nasa star canada bike score always goal maynard area player hockey wings center white
ANSWER:[/INST]""".strip()

answer = generate_answer(prompt, model, 'cuda', temp=0.1, n_token=100)

print(answer[0])


<s>[INST] <<SYS>> You are the assistant for text classification <<SYS>>
You will receive a TEXT, and you should answer "YES" or "NO" to the question: "Is it possible to determine a common topic for the TEXT or at least for the most part of the TEXT?". Please, make sure you to only return YES or NO and nothing more.
TEXT: april nasa star canada bike score always goal maynard area player hockey wings center white
ANSWER:[/INST]  NO.</s>


### Low-level code for text generation

In [ ]:
prompt = """<<SYS>> You are the assistant for text classification <</SYS>>
You will receive a TEXT, and you should answer "YES" or "NO" to the question: "Is it possible to determine a common topic for the TEXT or at least for the most part of the TEXT?". Please, make sure you to only return YES or NO and nothing more.

TEXT: april nasa star canada bike score always goal maynard area player hockey wings center white
ANSWER:""".strip()

print(prompt)

voc = tokenizer.get_vocab()
voc_rev = {v:k for k, v in voc.items()}

for i in range(1):
    inputs = tokenizer(prompt, return_tensors='pt', return_token_type_ids=False).to('cuda')
    logits = model.forward(**inputs).logits[0, -1, :]
    probs = torch.nn.functional.softmax(logits, dim=-1)
    next_token_id = torch.multinomial(probs.flatten(), num_samples=1)

    next_token = tokenizer.decode(next_token_id)
    print("next token", next_token)
    prompt += next_token


    sorted_probs, sorted_indices = torch.sort(probs, descending=True)
    top_tokens = sorted_indices[:5]
    print(f'Step #{i} candidates:')
    for t, p in zip(top_tokens, sorted_probs):
        t = voc_rev[t.item()]
        print(f"{t}: {p:.4f} ")

    print(f"\nChosen token: (next_token)", end="\n\n", flush=True)

<<SYS>> You are the assistant for text classification <</SYS>>
You will receive a TEXT, and you should answer "YES" or "NO" to the question: "Is it possible to determine a common topic for the TEXT or at least for the most part of the TEXT?". Please, make sure you to only return YES or NO and nothing more.

TEXT: april nasa star canada bike score always goal maynard area player hockey wings center white
ANSWER:
next token YES
Step #0 candidates:
▁YES: 0.3444 
▁NO: 0.2900 
<0x0A>: 0.2224 
▁: 0.0441 
▁?: 0.0169 

Chosen token: (next_token)



####Type II

In [ ]:
test_data = data.sample(frac=1, random_state=42)

In [ ]:
test_data

,task,agg_label
2424,pack variety com www http href stash pie grey ...,bad
3290,south island ship secret military rutgers brit...,bad
2575,piece beef jerky everything season break pill ...,bad
2042,mix cream ice pour strawberry stir pancake waf...,good
3717,window display screen color server application...,good
...,...,...
1130,engine model rocket band lock foot exhaust pre...,good
1294,font print character page printer vice cell bi...,good
860,delicious small dry hard smaller surprised liv...,bad
3507,thought end else stick cover boil air china bu...,bad


In [ ]:
test_data.iloc[1795]

task         launch entry satellite rule remark build europ...
agg_label                                                  bad
Name: 1872, dtype: object

In [ ]:
SYSTEM_PROMPT = "You are the assistant for text classification"
INSTRUCTION = "You will receive a TEXT, and you should answer 'YES' or 'NO' to the question: 'Is it possible to determine a common topic for the TEXT or at least for the most part of the TEXT?'. Please, make sure you to only return YES or NO and nothing more."

SAMPLE = test_data.iloc[785]["task"]
print(SAMPLE)

res = few_shot_pipeline(model, SYSTEM_PROMPT, INSTRUCTION, SAMPLE, device="cuda", n_token=30)

res

firearm weapon bill crime vote public laws handgun defense criminal washington federal member carry unite


'NO. There is no common topic that can be determined for the majority of the text. The text mentions various topics such as firearms,'

In [ ]:
torch.cuda.empty_cache()

In [ ]:
# Create file
RES_FILE_NAME = "data_qality_estim_llama_13b_4bit_type1.csv"
RES_FILE = DATA_DIR.joinpath(RES_FILE_NAME)
# # Remove old file
# if os.path.exists(RES_FILE):
#     os.remove(RES_FILE)
for i in tqdm(range(785, len(test_data))):
# for i in tqdm(range(20)):
    SAMPLE = test_data.iloc[i]["task"]
    res_true = test_data.iloc[i]["agg_label"]
    res_model = few_shot_pipeline(
        model,
        SYSTEM_PROMPT,
        INSTRUCTION,
        SAMPLE,
        device="cuda",
        temp=0.1,
        n_token=30
    )
    with open(RES_FILE, 'a') as file:
        file.write(f"{SAMPLE};{res_true};{res_model}\n")


100%|██████████| 3076/3076 [1:26:57<00:00,  1.70s/it]
